# Evaluate Base Model Endpoints using Azure AI Evaluation APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [ ]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install promptflow-tracing
%pip install marshmallow==3.23.3

### Parameters and imports

In [13]:
import pandas as pd
import random
import os
from pprint import pprint
# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"#This is the api version for the model for example 2024-05-01-preview
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "gpt-4o"#This is the deployment name for the model for example got-4o
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://xxxxx.openai.azure.com/"#This is end point for deployment for judging the model
os.environ["AZURE_SUBSCRIPTION_ID"] = "xxxxxx"#This is end point for deployment for judging the model
os.environ["AZURE_AI_FOUNDRY_RESOURCE_GROUP"] = "xxxxxx"#This is the resource group for AI Foundry
os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"] = "xxxxxx"#This is the project name for AI Foundry



In [ ]:
import os
print(os.environ['AZURE_OPENAI_ENDPOINT'])

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [15]:
env_var = {
    "gpt4": {
        "endpoint": "https://xxxxx.openai.azure.com/openai/deployments/xxxxx/chat/completions?api-version=xxxxx",
        "key": "xxxx",
    },
    "gpt35-turbo": {
        "endpoint": "https://xxxxx.openai.azure.com/openai/deployments/xxxxx/chat/completions?api-version=xxxxx",
        "key": "xxxx",
    }
}



Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [16]:
azure_ai_project = {
    "subscription_id": os.environ["AZURE_SUBSCRIPTION_ID"],
    "resource_group_name": os.environ["AZURE_AI_FOUNDRY_RESOURCE_GROUP"],
    "project_name": os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
}

## Model Endpoints
The following code demonstrates how to call various model endpoints, and is configured based on `env_var` set above. For any model in `env_var`, if you do not have that model deployed in your AI project, please comment it out. If you have a model that you would like to test that does not correspond with one of the types seen below, please include that type in the `__call__` function and create a helper function to call the model's endpoint via REST. 

In [ ]:
!pygmentize model_endpoints.py

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [ ]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [19]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [ ]:
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
)
from model_endpoints import ModelEndpoints

relevance_evaluator = RelevanceEvaluator(model_config)

models = [
    "gpt4",
    "gpt35-turbo"
]
results = {}
path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results[model] = evaluate(
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "relevance": relevance_evaluator,
        },
        evaluator_config={
            "relevance": {
                "column_mapping": {
                    "response": "${target.response}",
                    "context": "${data.context}",
                    "query": "${data.query}",
                },
            },
        },
    )

View the results

In [ ]:
results

In [ ]:
import numpy as np
pd = pd.DataFrame(results)
gpt4scores = [row['outputs.relevance.relevance'] for row in pd.gpt4.rows]
np.mean(gpt4scores)
gpt35_turbo_scores = [row['outputs.relevance.relevance'] for row in pd['gpt35-turbo'].rows]
np.mean(gpt35_turbo_scores)

print("GPT-4 Mean Relevance Score: ", np.mean(gpt4scores))
print("GPT-35-Turbo Mean Relevance Score: ", np.mean(gpt35_turbo_scores))